<a href="https://colab.research.google.com/github/NigelWilliamUOP/AI-Scientist-fork/blob/main/memecoin_rugpull_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install deps (Colab)
!pip -q install networkx pandas numpy matplotlib


In [3]:
#@title Upload `network.graphml`, `abm_shares_observed_v2.yaml`, `bot_edges_config.yaml`
from google.colab import files
import io, os, pandas as pd

uploaded = files.upload()  # pick: network.graphml, abm_shares_observed_v2.yaml, bot_edges_config.yaml

# Save with expected names if needed
rename_map = {
    'network.graphml': 'network.graphml',
    'abm_shares_observed_v2.yaml': 'abm_shares_observed_v2.yaml',
    'bot_edges_config.yaml': 'bot_edges_config.yaml'
}
for k, v in uploaded.items():
    name = os.path.basename(k)
    tgt = rename_map.get(name, name)
    with open(tgt, 'wb') as f:
        f.write(v)
print("Uploaded files:", list(uploaded.keys()))


Saving abm_shares_observed_v2.yaml to abm_shares_observed_v2.yaml
Saving bot_edges_config.yaml to bot_edges_config.yaml
Saving network.graphml to network (1).graphml
Uploaded files: ['abm_shares_observed_v2.yaml', 'bot_edges_config.yaml', 'network (1).graphml']


In [5]:
#@title Helpers: YAML-lite, BCa, Hedges' g
import math, numpy as np

def load_shares_yaml(path='abm_shares_observed_v2.yaml'):
    txt = open(path, 'r', encoding='utf-8').read().splitlines()
    personas = []; current=None
    for line in txt:
        s=line.strip()
        if s.startswith('- name:'):
            name = s.split(':',1)[1].strip()
            current={'name': name}; personas.append(current)
        elif 'share:' in s and current is not None:
            current['share'] = float(s.split(':',1)[1].strip())
    share_map = {d['name']: d['share'] for d in personas if 'share' in d}
    # remove Bot/Amplifier; renormalise
    share_map.pop('Bot/Amplifier', None)
    tot = sum(share_map.values()) or 1.0
    return {k: v/tot for k,v in share_map.items()}

def load_botcfg_yaml(path='bot_edges_config.yaml'):
    params={}; promoter_classes=[]; section=None
    for raw in open(path,'r',encoding='utf-8'):
        s=raw.strip()
        if not s or s.startswith('#'): continue
        if s.startswith('parameters:'): section='parameters'; continue
        if s.startswith('promoter_classes:'): section='promoter_classes'; continue
        if section=='parameters' and ':' in s and not s.startswith('-'):
            k,v = s.split(':',1); v=v.split('#',1)[0].strip()
            if v.lower() in ('true','false'):
                v=(v.lower()=='true')
            else:
                try: v=float(v)
                except: pass
            params[k.strip()]=v
        elif section=='promoter_classes' and s.startswith('-'):
            promoter_classes.append(s[1:].split('#',1)[0].strip())
    return params, promoter_classes

def hedges_g(x, y):
    x=np.asarray(x, dtype=float); y=np.asarray(y, dtype=float)
    n1, n2 = len(x), len(y)
    m1, m2 = x.mean(), y.mean()
    v1, v2 = x.var(ddof=1), y.var(ddof=1)
    sp2 = ((n1-1)*v1 + (n2-1)*v2) / (n1+n2-2)
    J = 1 - (3 / (4*(n1+n2) - 9))
    return float(J * (m2 - m1) / ((sp2+1e-12)**0.5))

def norm_ppf(p):
    # Acklam approximation (no SciPy needed)
    a=[-3.969683028665376e+01,2.209460984245205e+02,-2.759285104469687e+02,1.383577518672690e+02,-3.066479806614716e+01,2.506628277459239e+00]
    b=[-5.447609879822406e+01,1.615858368580409e+02,-1.556989798598866e+02,6.680131188771972e+01,-1.328068155288572e+01]
    c=[-7.784894002430293e-03,-3.223964580411365e-01,-2.400758277161838e+00,-2.549732539343734e+00,4.374664141464968e+00,2.938163982698783e+00]
    d=[7.784695709041462e-03,3.224671290700398e-01,2.445134137142996e+00,3.754408661907416e+00]
    plow=0.02425; phigh=1-plow
    p=float(np.clip(p,1e-12,1-1e-12))
    if p<plow:
        q=math.sqrt(-2*math.log(p))
        return (((((c[0]*q+c[1])*q+c[2])*q+c[3])*q+c[4])*q+c[5])/((((d[0]*q+d[1])*q+d[2])*q+d[3])*q+1)
    if p>phigh:
        q=math.sqrt(-2*math.log(1-p))
        return -(((((c[0]*q+c[1])*q+c[2])*q+c[3])*q+c[4])*q+c[5])/((((d[0]*q+d[1])*q+d[2])*q+d[3])*q+1)
    q=p-0.5; r=q*q
    return (((((a[0]*r+a[1])*r+a[2])*r+a[3])*r+a[4])*r+a[5])*q/((((b[0]*r+b[1])*r+b[2])*r+b[3])*r+b[4]*r+1)

def Phi(z):
    return 0.5*(1+math.erf(z/math.sqrt(2)))

def bca_two_sample_effect(x, y, stat_fn=hedges_g, B=2500, alpha=(0.025,0.975), seed=123):
    rng = np.random.default_rng(seed)
    x=np.asarray(x, float); y=np.asarray(y, float)
    theta = stat_fn(x,y)
    n1, n2 = len(x), len(y)
    boots = np.empty(B, dtype=float)
    for b in range(B):
        xb = rng.choice(x, size=n1, replace=True)
        yb = rng.choice(y, size=n2, replace=True)
        boots[b] = stat_fn(xb, yb)
    boots.sort()
    prop_less = (boots < theta).mean()
    z0 = norm_ppf(prop_less)
    # jackknife over observations
    thetas_j=[]
    for i in range(n1): thetas_j.append(stat_fn(np.delete(x,i), y))
    for j in range(n2): thetas_j.append(stat_fn(x, np.delete(y,j)))
    thetas_j=np.asarray(thetas_j)
    tdot=thetas_j.mean()
    num=np.sum((tdot - thetas_j)**3)
    den=6.0*(np.sum((tdot - thetas_j)**2)**1.5 + 1e-18)
    a = num/den if den!=0 else 0.0
    qs=[]
    for q in alpha:
        zq=norm_ppf(q)
        adj = z0 + (z0+zq)/max(1e-18, (1 - a*(z0+zq)))
        qs.append(Phi(adj))
    lo=np.quantile(boots, qs[0]); hi=np.quantile(boots, qs[1])
    return float(theta), float(lo), float(hi)

In [6]:
#@title Build empirical adjacency from GraphML (largest component; degree-proportional down-sample to N=500)
import networkx as nx
import pandas as pd
import numpy as np

GRAPHML_PATH = 'network.graphml'  # change if needed
N_target = 500  #@param {type:"integer"}

G = nx.read_graphml(GRAPHML_PATH)
if G.is_directed():
    G = G.to_undirected()
G = nx.Graph(G)
G.remove_edges_from(nx.selfloop_edges(G))

# Largest connected component
comp = max(nx.connected_components(G), key=len)
H = G.subgraph(comp).copy()

# Degree-proportional down-sampling (if needed)
if H.number_of_nodes() > N_target:
    nodes = list(H.nodes())
    degs = np.array([H.degree(n) for n in nodes], dtype=float)
    w = degs / degs.sum() if degs.sum()>0 else np.ones_like(degs)/len(degs)
    sel = set(np.random.default_rng(20240915).choice(nodes, size=N_target, replace=False, p=w))
    H = H.subgraph(sel).copy()

# Relabel 0..N-1 and materialise adjacency as list[set]
H = nx.convert_node_labels_to_integers(H, ordering="decreasing degree")
N_emp = H.number_of_nodes()
G_adj = [set() for _ in range(N_emp)]
for u,v in H.edges():
    G_adj[u].add(v); G_adj[v].add(u)

deg = np.array([len(G_adj[i]) for i in range(N_emp)])
deg_summary = dict(
    N_nodes=int(N_emp),
    N_edges=int(H.number_of_edges()),
    deg_min=int(deg.min()) if len(deg) else 0,
    deg_med=float(np.median(deg)) if len(deg) else 0.0,
    deg_mean=float(deg.mean()) if len(deg) else 0.0,
    deg_max=int(deg.max()) if len(deg) else 0
)
pd.DataFrame([deg_summary])


,N_nodes,N_edges,deg_min,deg_med,deg_mean,deg_max
0,500,2402,0,4.0,9.608,197


In [7]:
#@title ABM models (diffusion and rugpull) on fixed adjacency
import random, math, numpy as np

THETA = {
    "FOMO Novice": 0.25, "Degen Speculator": 0.35, "Community-Driven": 0.45,
    "Influencer/Promoter": 0.20, "Whale/Liquidity Mover": 0.40,
    "Insider/Developer": 0.30, "Contrarian Skeptic": 0.60, "DYOR Enforcer": 0.50,
    "Cautious Lurker": 0.55, "Shill Ring/Testimonial": 0.25, "Compliance Educator": 0.55,
    "Other": 0.45,
}
SENS = {
    "FOMO Novice": 0.4, "Degen Speculator": 0.5, "Community-Driven": 0.45,
    "Influencer/Promoter": 0.2, "Whale/Liquidity Mover": 0.6,
    "Insider/Developer": 0.8, "Contrarian Skeptic": 0.9, "DYOR Enforcer": 0.8,
    "Cautious Lurker": 0.7, "Shill Ring/Testimonial": 0.35, "Compliance Educator": 0.85,
    "Other": 0.5,
}
EXIT_TH = {
    "FOMO Novice": 0.65, "Degen Speculator": 0.7, "Community-Driven": 0.6,
    "Influencer/Promoter": 0.9, "Whale/Liquidity Mover": 0.6,
    "Insider/Developer": 0.5, "Contrarian Skeptic": 0.45, "DYOR Enforcer": 0.55,
    "Cautious Lurker": 0.5, "Shill Ring/Testimonial": 0.85, "Compliance Educator": 0.5,
    "Other": 0.6,
}

class PersonaModelFixed:
    def __init__(self, G_adj, shares, promoter_classes, kappa=1.0, lam=5.4,
                 seed_share=0.01, t_max=150, rng_seed=None):
        self.G = G_adj; self.N = len(G_adj)
        self.seed_share=seed_share; self.t_max=t_max
        self.rng = random.Random(rng_seed)
        # personas
        self.persona = self._sample_personas(shares)
        self.threshold = [THETA.get(p,0.45) for p in self.persona]
        self.promoter = [p in promoter_classes or p=="Shill Ring/Testimonial" for p in self.persona]
        # amplification weights
        promo_ids = [i for i,f in enumerate(self.promoter) if f]
        db = np.zeros(self.N, dtype=float)
        for j in promo_ids:
            db[j] = np.random.poisson(lam)
        maxdb = db[promo_ids].max() if promo_ids else 1.0
        bj = (db/maxdb) if maxdb>0 else db
        self.A = 1.0 + kappa * bj
        # seeding
        self.adopted = [False]*self.N
        n_seed = max(1, int(round(seed_share*self.N)))
        seeded=set()
        if promo_ids:
            seeded.add(self.rng.choice(promo_ids))
        while len(seeded)<n_seed:
            seeded.add(self.rng.randrange(self.N))
        for i in seeded: self.adopted[i]=True
        self.t=0; self.new_adoptions=0; self.peak_time=0; self.peak_size=0
        self.adopt_time=[0 if self.adopted[i] else None for i in range(self.N)]

    def _sample_personas(self, shares):
        names=list(shares.keys()); probs=np.array([shares[n] for n in names], float)
        counts=np.random.multinomial(self.N, probs, size=1)[0]
        personas=[]
        for name,c in zip(names, counts): personas += [name]*int(c)
        while len(personas)<self.N: personas.append("Other")
        if len(personas)>self.N: personas=personas[:self.N]
        self.rng.shuffle(personas); return personas

    def step(self):
        nxt=self.adopted.copy(); inc=0
        for i in range(self.N):
            if self.adopted[i]: continue
            neigh=self.G[i]
            if not neigh: continue
            denom=0.0; numer=0.0
            for j in neigh:
                w = self.A[j] if self.promoter[j] else 1.0
                denom += w
                if self.adopted[j]: numer += w
            share = numer/denom if denom>0 else 0.0
            if share >= self.threshold[i]:
                nxt[i]=True; inc+=1
        if inc>self.peak_size: self.peak_size=inc; self.peak_time=self.t
        self.new_adoptions=inc; self.adopted=nxt; self.t+=1

    def run(self):
        stable=0
        while self.t<self.t_max and stable<2:
            self.step()
            stable = stable+1 if self.new_adoptions==0 else 0

    def adopted_fraction(self):
        return sum(1 for a in self.adopted if a)/self.N

class RugpullModelFixed(PersonaModelFixed):
    def __init__(self, *args, alpha_up=0.8, gamma_down=0.9,
                 t_lock=100, h0=0.0005, h1=0.03, w=8, salvage=0.08, **kwargs):
        super().__init__(*args, **kwargs)
        self.alpha=alpha_up; self.gamma=gamma_down
        self.t_lock=t_lock; self.h0=h0; self.h1=h1; self.w=w; self.salvage=salvage
        self.rugged=False; self.t_rug=None
        self.P=[1.0]
        self.sold=[False]*self.N; self.trapped=[False]*self.N
        self.sell_time=[None]*self.N
        self.adopt_price=[1.0 if (self.adopt_time[i]==0) else None for i in range(self.N)]
        self.sell_price=[None]*self.N
        self.new_exits=0

    def hazard(self, t):
        x=(t-self.t_lock)/self.w
        sig=1/(1+math.exp(-x))
        return min(0.5, self.h0 + self.h1*sig)

    def signal(self, t):
        x=(t-self.t_lock+5)/self.w
        return max(0.0, min(1.0, 1/(1+math.exp(-x))))

    def step(self):
        if self.rugged:
            self.P.append(max(self.P[-1]*0.95, self.salvage*self.P[0])); self.t+=1; return
        # rug?
        if random.random() < self.hazard(self.t):
            self.rugged=True; self.t_rug=self.t
            self.P.append(max(self.P[-1]*0.1, self.salvage*self.P[0]))
            for i in range(self.N):
                if self.adopted[i] and not self.sold[i]:
                    self.trapped[i]=True; self.sold[i]=True; self.sell_time[i]=self.t; self.sell_price[i]=self.P[-1]
            self.t+=1; return
        # exits
        s=self.signal(self.t); exits=0
        for i in range(self.N):
            if self.adopted[i] and (not self.sold[i]):
                reluctance = 0.85 if self.promoter[i] else 1.0
                perceived = s * SENS.get(self.persona[i],0.5) * reluctance
                if perceived >= EXIT_TH.get(self.persona[i],0.6):
                    self.sold[i]=True; self.sell_time[i]=self.t; self.sell_price[i]=self.P[-1]; exits+=1
        self.new_exits=exits
        # adoptions
        nxt=self.adopted.copy(); inc=0
        for i in range(self.N):
            if self.adopted[i]: continue
            neigh=self.G[i]
            if not neigh: continue
            denom=0.0; numer=0.0
            for j in neigh:
                w=self.A[j] if self.promoter[j] else 1.0
                denom+=w
                if self.adopted[j]: numer+=w
            share = numer/denom if denom>0 else 0.0
            if share >= THETA.get(self.persona[i],0.45):
                nxt[i]=True; inc+=1
        if inc>self.peak_size: self.peak_size=inc; self.peak_time=self.t
        self.adopted=nxt
        for i in range(self.N):
            if self.adopted[i] and self.adopt_time[i] is None:
                self.adopt_time[i]=self.t; self.adopt_price[i]=self.P[-1]
        # price
        growth = self.alpha * (inc / self.N)
        pressure = self.gamma * (self.new_exits / self.N)
        dlogP = growth - pressure
        self.P.append(max(0.01, self.P[-1] * math.exp(dlogP)))
        self.t+=1

    def run(self, t_max=200):
        stable=0
        while self.t<t_max and stable<3:
            self.step()
            if self.rugged: stable += 1
            else: stable = stable+1 if (self.new_exits==0 and self.new_adoptions==0) else 0

    def results(self):
        losses=[]
        for i in range(self.N):
            if self.adopt_time[i] is None: continue
            buy = self.adopt_price[i] if self.adopt_price[i] is not None else self.P[0]
            sell = self.sell_price[i] if self.sell_price[i] is not None else self.P[-1]
            loss = max(0.0, buy - sell)
            losses.append((self.persona[i], loss, self.trapped[i]))
        return losses


In [8]:
#@title κ comparison (κ=0 vs κ=1) + BCa CIs for peak time and final adoption
import pandas as pd, numpy as np

shares = load_shares_yaml('abm_shares_observed_v2.yaml')
botparams, promoter_classes = load_botcfg_yaml('bot_edges_config.yaml')
LAM = float(botparams.get('bot_degree_poisson_lambda', 5.4))

def run_kappa_empirical(G_adj, reps=60, seed_base=20240915):
    rows=[]
    for kappa in [0.0, 1.0]:
        for r in range(reps):
            seed = (seed_base * 73856093 + r * 19349663 + int(kappa*1e6)) % (2**32-1)
            m = PersonaModelFixed(G_adj, shares, promoter_classes, kappa=kappa, lam=LAM,
                                  seed_share=0.01, t_max=150, rng_seed=seed)
            m.run()
            rows.append({
                "kappa": kappa, "rep": r,
                "peak_time": m.peak_time,
                "peak_size": m.peak_size,
                "adopted_fraction": m.adopted_fraction()
            })
    return pd.DataFrame(rows)

df_runs = run_kappa_empirical(G_adj, reps=60)
cond = df_runs.groupby("kappa").agg(
    runs=("rep","count"),
    peak_time_mean=("peak_time","mean"),
    peak_time_sd=("peak_time","std"),
    adopted_mean=("adopted_fraction","mean"),
    adopted_sd=("adopted_fraction","std"),
    peak_size_mean=("peak_size","mean"),
).reset_index()
display(cond)

rows=[]
for metric in ["peak_time","adopted_fraction"]:
    x = df_runs[df_runs.kappa==0.0][metric].to_numpy()
    y = df_runs[df_runs.kappa==1.0][metric].to_numpy()
    g, lo, hi = bca_two_sample_effect(x, y, stat_fn=hedges_g, B=2500)
    rows.append({"metric": metric, "hedges_g_point": round(g,3), "ci_lo": round(lo,3), "ci_hi": round(hi,3),
                 "n_k0": len(x), "n_k1": len(y)})
eff = pd.DataFrame(rows)
display(eff)

# Persist outputs
os.makedirs("out_emp_kappa", exist_ok=True)
df_runs.to_csv("out_emp_kappa/runs_summary_kappa_comp.csv", index=False)
cond.to_csv("out_emp_kappa/condition_summaries_empirical.csv", index=False)
eff.to_csv("out_emp_kappa/effect_sizes_bca_empirical.csv", index=False)
print("Saved to out_emp_kappa/")


,kappa,runs,peak_time_mean,peak_time_sd,adopted_mean,adopted_sd,peak_size_mean
0,0.0,60,0.000000,0.000000,0.017167,0.013984,2.866667
1,1.0,60,0.016667,0.129099,0.014967,0.004751,2.200000


,metric,hedges_g_point,ci_lo,ci_hi,n_k0,n_k1
0,peak_time,0.181,0.181,0.181,60,60
1,adopted_fraction,-0.209,-0.200,-0.200,60,60


Saved to out_emp_kappa/


In [9]:
#@title Rugpull on empirical network (persona losses and trapped rate)
import pandas as pd, numpy as np

KAPPA_BASE = float(botparams.get('amplification_kappa', 1.0))

def run_rugpull_empirical(G_adj, reps=50, seed_base=424242):
    runs=[]; per=[]
    for r in range(reps):
        seed = (seed_base * 73471 + r * 271) % (2**32-1)
        m = RugpullModelFixed(G_adj, shares, promoter_classes, kappa=KAPPA_BASE, lam=LAM,
                              seed_share=0.01, t_max=200, rng_seed=seed)
        m.run(t_max=200)
        trapped_rate = sum(1 for i in range(m.N) if m.trapped[i])/m.N
        runs.append({
            "rep": r,
            "rug_time": (m.t_rug if m.t_rug is not None else -1),
            "final_price": m.P[-1],
            "trapped_rate": trapped_rate,
            "peak_time": m.peak_time,
            "final_adoption": m.adopted_fraction(),
        })
        losses = m.results()
        byp={}
        for pers, loss, trapped in losses:
            d=byp.setdefault(pers, {"n":0,"loss_sum":0.0,"trapped":0})
            d["n"]+=1; d["loss_sum"]+=loss; d["trapped"]+=int(trapped)
        for pers,d in byp.items():
            per.append({
                "rep": r, "persona": pers, "n_adopters": d["n"],
                "mean_loss": d["loss_sum"]/d["n"] if d["n"]>0 else 0.0,
                "trapped_share": d["trapped"]/d["n"] if d["n"]>0 else 0.0
            })
    return pd.DataFrame(runs), pd.DataFrame(per)

df_rug, df_rugp = run_rugpull_empirical(G_adj, reps=50)

rug_summary = pd.DataFrame({
    "runs":[len(df_rug)],
    "rug_time_median":[df_rug["rug_time"].median()],
    "trapped_rate_mean":[df_rug["trapped_rate"].mean()],
    "final_adoption_mean":[df_rug["final_adoption"].mean()],
    "final_price_mean":[df_rug["final_price"].mean()],
    "peak_time_median":[df_rug["peak_time"].median()],
})
display(rug_summary)

os.makedirs("out_emp_rugpull", exist_ok=True)
df_rug.to_csv("out_emp_rugpull/runs_rugpull.csv", index=False)
df_rugp.to_csv("out_emp_rugpull/persona_rugpull_aggregated.csv", index=False)
print("Saved to out_emp_rugpull/")


,runs,rug_time_median,trapped_rate_mean,final_adoption_mean,final_price_mean,peak_time_median
0,50,-1.0,0.0,0.01504,1.004047,0.0


Saved to out_emp_rugpull/


In [10]:
#@title Rugpull on empirical network (persona losses and trapped rate)
import pandas as pd, numpy as np

KAPPA_BASE = float(botparams.get('amplification_kappa', 1.0))

def run_rugpull_empirical(G_adj, reps=50, seed_base=424242):
    runs=[]; per=[]
    for r in range(reps):
        seed = (seed_base * 73471 + r * 271) % (2**32-1)
        m = RugpullModelFixed(G_adj, shares, promoter_classes, kappa=KAPPA_BASE, lam=LAM,
                              seed_share=0.01, t_max=200, rng_seed=seed)
        m.run(t_max=200)
        trapped_rate = sum(1 for i in range(m.N) if m.trapped[i])/m.N
        runs.append({
            "rep": r,
            "rug_time": (m.t_rug if m.t_rug is not None else -1),
            "final_price": m.P[-1],
            "trapped_rate": trapped_rate,
            "peak_time": m.peak_time,
            "final_adoption": m.adopted_fraction(),
        })
        losses = m.results()
        byp={}
        for pers, loss, trapped in losses:
            d=byp.setdefault(pers, {"n":0,"loss_sum":0.0,"trapped":0})
            d["n"]+=1; d["loss_sum"]+=loss; d["trapped"]+=int(trapped)
        for pers,d in byp.items():
            per.append({
                "rep": r, "persona": pers, "n_adopters": d["n"],
                "mean_loss": d["loss_sum"]/d["n"] if d["n"]>0 else 0.0,
                "trapped_share": d["trapped"]/d["n"] if d["n"]>0 else 0.0
            })
    return pd.DataFrame(runs), pd.DataFrame(per)

df_rug, df_rugp = run_rugpull_empirical(G_adj, reps=50)

rug_summary = pd.DataFrame({
    "runs":[len(df_rug)],
    "rug_time_median":[df_rug["rug_time"].median()],
    "trapped_rate_mean":[df_rug["trapped_rate"].mean()],
    "final_adoption_mean":[df_rug["final_adoption"].mean()],
    "final_price_mean":[df_rug["final_price"].mean()],
    "peak_time_median":[df_rug["peak_time"].median()],
})
display(rug_summary)

os.makedirs("out_emp_rugpull", exist_ok=True)
df_rug.to_csv("out_emp_rugpull/runs_rugpull.csv", index=False)
df_rugp.to_csv("out_emp_rugpull/persona_rugpull_aggregated.csv", index=False)
print("Saved to out_emp_rugpull/")


,runs,rug_time_median,trapped_rate_mean,final_adoption_mean,final_price_mean,peak_time_median
0,50,-1.0,0.0,0.01544,1.004373,0.0


Saved to out_emp_rugpull/


In [ ]:
#@title Assemble combined tables (ER, WS, Empirical) for the Results
from google.colab import files
import pandas as pd, os

# If needed, upload the three files below; else skip the upload dialogue
print("If these are not in the working dir, upload now:")
print("- abm_run_kappa_comp/condition_summaries.csv")
print("- inference_outputs/effect_sizes_bca_kappa.csv")
print("- abm_run_rugpull/runs_rugpull.csv")
_ = files.upload()

# Read ER/WS
erws_kappa_cond = pd.read_csv("abm_run_kappa_comp/condition_summaries.csv")
erws_kappa_bca  = pd.read_csv("inference_outputs/effect_sizes_bca_kappa.csv")
erws_rug_runs   = pd.read_csv("abm_run_rugpull/runs_rugpull.csv")

# Read empirical we just produced
emp_kappa_cond  = pd.read_csv("out_emp_kappa/condition_summaries_empirical.csv")
emp_kappa_bca   = pd.read_csv("out_emp_kappa/effect_sizes_bca_empirical.csv")
emp_rug_runs    = pd.read_csv("out_emp_rugpull/runs_rugpull.csv")

emp_kappa_cond["graph"]="empirical"
emp_kappa_bca["graph"]="empirical"

kappa_cond_all = pd.concat([erws_kappa_cond, emp_kappa_cond], ignore_index=True)
kappa_bca_all  = pd.concat([erws_kappa_bca, emp_kappa_bca], ignore_index=True)

def summarise_rug(df):
    return pd.DataFrame({
        "runs":[len(df)],
        "rug_time_median":[df["rug_time"].median()],
        "trapped_rate_mean":[df["trapped_rate"].mean()],
        "final_adoption_mean":[df["final_adoption"].mean()],
        "final_price_mean":[df["final_price"].mean()],
        "peak_time_median":[df["peak_time"].median()],
    })

rows=[]
for g, sub in erws_rug_runs.groupby("graph"):
    s=summarise_rug(sub); s["graph"]=g; rows.append(s)
s=summarise_rug(emp_rug_runs); s["graph"]="empirical"; rows.append(s)
rug_all = pd.concat(rows, ignore_index=True)

os.makedirs("results", exist_ok=True)
kappa_cond_all.to_csv("results/kappa_condition_summaries_all.csv", index=False)
kappa_bca_all.to_csv("results/kappa_effect_sizes_bca_all.csv", index=False)
rug_all.to_csv("results/rugpull_summary_all.csv", index=False)

print("Wrote results/kappa_condition_summaries_all.csv, results/kappa_effect_sizes_bca_all.csv, results/rugpull_summary_all.csv")
